RFF = really fucking fast :)

In [1]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_moons
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
np.random.seed(2)

In [3]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.interpolate import PchipInterpolator, CubicSpline
from joblib import Parallel, delayed

class SMPAEstimator(BaseEstimator, ClassifierMixin):
    def __init__(self, learning_rate=0.005, epochs=50, random_state=None, verbose=False,
                 lambda_scaling='log', patience=5, decay_factor=0.5, min_learning_rate=1e-6,
                 n_control_points=5, smoothing_factor=0.1, spline_type='pchip', early_stop_patience=20):
        self.learning_rate = learning_rate
        self.initial_learning_rate = learning_rate
        self.epochs = epochs
        self.random_state = random_state
        self.verbose = verbose
        self.lambda_scaling = lambda_scaling
        self.patience = patience
        self.decay_factor = decay_factor
        self.min_learning_rate = min_learning_rate
        self.n_control_points = n_control_points
        self.smoothing_factor = smoothing_factor
        self.spline_type = spline_type
        self.early_stop_patience = early_stop_patience

        if lambda_scaling not in ['log', 'sqrt', 'none']:
            raise ValueError("lambda_scaling must be one of 'log', 'sqrt', or 'none'")
        if spline_type not in ['cubic', 'pchip']:
            raise ValueError("spline_type must be one of 'cubic' or 'pchip'")

        if random_state is not None:
            np.random.seed(random_state)

    def _calculate_class_means(self, X, y):
        mask_1 = y == 1
        self.m1 = X[mask_1].mean(axis=0)
        self.m0 = X[~mask_1].mean(axis=0)

    def _initialize_control_points(self, X):
        x_min, x_max = X[:, 0].min(), X[:, 0].max()
        y_min, y_max = X[:, 1].min(), X[:, 1].max()
        x_range = x_max - x_min
        y_range = y_max - y_min
        x_min_extended = x_min - 0.2 * x_range
        x_max_extended = x_max + 0.2 * x_range
        self.control_x = np.linspace(x_min_extended, x_max_extended, self.n_control_points)
        y_mid = (self.m0[1] + self.m1[1]) / 2
        self.control_y = np.random.uniform(y_mid - y_range * 0.05, y_mid + y_range * 0.05, self.n_control_points)
        if self.n_control_points > 2:
            self.control_y[0] = self.control_y[1]
            self.control_y[-1] = self.control_y[-2]

    def _fit_spline(self):
        if not np.all(np.diff(self.control_x) > 0):
            raise ValueError("control_x must be strictly increasing for spline fitting")
        if self.spline_type == 'cubic':
            self.spline = CubicSpline(self.control_x, self.control_y, bc_type='clamped')
        else:
            self.spline = PchipInterpolator(self.control_x, self.control_y)

    def _calculate_displacement(self, X):
        return X[:, 1] - self.spline(X[:, 0])

    def _update_pseudo_labels(self, X, y):
        m1_displacement = self._calculate_displacement(self.m1.reshape(1, -1))[0]
        self.class_1_pseudo = 1 if m1_displacement > 0 else -1
        self.class_0_pseudo = -self.class_1_pseudo
        return np.where(y == 1, self.class_1_pseudo, self.class_0_pseudo)

    def fit(self, X, y):
        if not set(np.unique(y)).issubset({0, 1}) or X.shape[1] != 2:
            raise ValueError("Labels must be 0 and 1, and X must be 2D")

        X = np.asarray(X)
        y = np.asarray(y)
        self._calculate_class_means(X, y)
        self._initialize_control_points(X)
        self._fit_spline()

        best_error = float('inf')
        best_control_x = None
        best_control_y = None
        best_class_1_pseudo = None
        patience_counter = 0
        early_stop_counter = 0
        current_learning_rate = self.initial_learning_rate

        indices_class_0 = np.where(y == 0)[0]
        indices_class_1 = np.where(y == 1)[0]

        for epoch in range(self.epochs):
            pseudo_labels = self._update_pseudo_labels(X, y)
            displacements = self._calculate_displacement(X)
            errors = (displacements * pseudo_labels <= 0)
            error_count = np.sum(errors)

            if self.verbose and epoch % 5 == 0:
                print(f"Epoch {epoch}: Errors = {error_count}, LR = {current_learning_rate:.6f}")

            if error_count < best_error:
                best_error = error_count
                best_control_x = self.control_x.copy()
                best_control_y = self.control_y.copy()
                best_class_1_pseudo = self.class_1_pseudo
                patience_counter = 0
                early_stop_counter = 0
            else:
                patience_counter += 1
                early_stop_counter += 1
                if patience_counter >= self.patience:
                    current_learning_rate = max(current_learning_rate * self.decay_factor, self.min_learning_rate)
                    patience_counter = 0
                    if current_learning_rate == self.min_learning_rate:
                        if self.verbose:
                            print(f"Min learning rate reached at epoch {epoch}")
                        break
                if early_stop_counter >= self.early_stop_patience:
                    if self.verbose:
                        print(f"Early stopping at epoch {epoch}")
                    break

            if errors.any():
                error_indices = np.where(errors)[0]
                np.random.shuffle(error_indices)

                for idx in error_indices:
                    d = X[idx]
                    distances = np.abs(self.control_x - d[0])
                    nearest_idx = np.argmin(distances)
                    distance = distances[nearest_idx]

                    lmbda = {'log': np.log1p(distance), 'sqrt': np.sqrt(distance), 'none': distance}[self.lambda_scaling]
                    margin = max(0.1, min(1.0, lmbda * 0.2))

                    opp_indices = indices_class_0 if y[idx] == 1 else indices_class_1
                    opp_displacements = displacements[opp_indices]
                    opp_labels = pseudo_labels[opp_indices]
                    correct_opp = opp_indices[opp_displacements * opp_labels > margin]

                    if len(correct_opp) > 0:
                        n_random = max(1, min(len(correct_opp) // 5, 5))
                        random_correct = np.random.choice(correct_opp, size=n_random, replace=False)
                        random_avg_opp = np.mean(X[random_correct], axis=0)
                        # Only update control_y, keep control_x fixed
                        step_x = 0  # Lock control_x updates
                        step_y = (random_avg_opp[1] - self.control_y[nearest_idx]) * current_learning_rate / (1 + lmbda)
                    else:
                        step_x = 0
                        step_y = -pseudo_labels[idx] * margin * current_learning_rate

                    # Apply updates iteratively
                    self.control_x[nearest_idx] += step_x  # This will be 0, but keeping structure
                    self.control_y[nearest_idx] += step_y

                    # Boundary handling for control_y
                    if self.n_control_points > 2:
                        self.control_y[0] = self.control_y[1]
                        self.control_y[-1] = self.control_y[-2]

                    # Refit spline after each update
                    self._fit_spline()
                    pseudo_labels = self._update_pseudo_labels(X, y)
                    displacements = self._calculate_displacement(X)

        self.control_x = best_control_x
        self.control_y = best_control_y
        self._fit_spline()
        self.class_1_pseudo = best_class_1_pseudo
        return self

    def predict(self, X):
        X = np.asarray(X)
        displacements = self._calculate_displacement(X)
        return np.where(displacements > 0, 1 if self.class_1_pseudo > 0 else 0, 0 if self.class_1_pseudo > 0 else 1)

    def predict_proba(self, X):
        X = np.asarray(X)
        displacements = self._calculate_displacement(X)
        raw_probs = 1 / (1 + np.exp(-displacements * self.class_1_pseudo * 0.5))
        return np.column_stack([1 - raw_probs, raw_probs]) if self.class_1_pseudo > 0 else np.column_stack([raw_probs, 1 - raw_probs])


class EnsembleSMPA(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators=5, bootstrap=True, bootstrap_features=False, 
                 max_features=1.0, max_samples=1.0, random_state=None, verbose=False,
                 learning_rate=0.005, epochs=50, lambda_scaling='log', patience=5, 
                 decay_factor=0.5, min_learning_rate=1e-6, n_control_points=5, 
                 smoothing_factor=0.1, spline_type='pchip', early_stop_patience=301):
        self.n_estimators = n_estimators
        self.bootstrap = bootstrap
        self.bootstrap_features = bootstrap_features
        self.max_features = max_features
        self.max_samples = max_samples
        self.random_state = random_state
        self.verbose = verbose
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.lambda_scaling = lambda_scaling
        self.patience = patience
        self.decay_factor = decay_factor
        self.min_learning_rate = min_learning_rate
        self.n_control_points = n_control_points
        self.smoothing_factor = smoothing_factor
        self.spline_type = spline_type
        self.early_stop_patience = early_stop_patience
        self.rng = np.random.RandomState(random_state)

    def _fit_estimator(self, X, y, sample_indices, feature_indices, seed):
        estimator = SMPAEstimator(
            learning_rate=self.learning_rate, epochs=self.epochs, random_state=seed,
            verbose=self.verbose, lambda_scaling=self.lambda_scaling, patience=self.patience,
            decay_factor=self.decay_factor, min_learning_rate=self.min_learning_rate,
            n_control_points=self.n_control_points, smoothing_factor=self.smoothing_factor,
            spline_type=self.spline_type, early_stop_patience=self.early_stop_patience
        )
        return {'estimator': estimator.fit(X[sample_indices][:, feature_indices], y[sample_indices]),
                'sample_indices': sample_indices, 'feature_indices': feature_indices}

    def fit(self, X, y):
        if not set(np.unique(y)).issubset({0, 1}) or X.shape[1] != 2:
            raise ValueError("Labels must be 0 and 1, and X must be 2D")

        X = np.asarray(X)
        y = np.asarray(y)
        n_samples, n_features = X.shape
        max_samples = int(self.max_samples * n_samples) if isinstance(self.max_samples, float) else min(self.max_samples, n_samples)
        max_features = max(1, int(self.max_features * n_features)) if isinstance(self.max_features, float) else min(self.max_features, n_features)

        sample_indices_list = [self.rng.choice(n_samples, max_samples, replace=True) if self.bootstrap else np.arange(n_samples) 
                               for _ in range(self.n_estimators)]
        feature_indices_list = [self.rng.choice(n_features, max_features, replace=False) if self.bootstrap_features else np.arange(n_features) 
                                for _ in range(self.n_estimators)]
        seeds = [self.rng.randint(0, 10000) if self.random_state is not None else None for _ in range(self.n_estimators)]

        self.estimators_ = Parallel(n_jobs=-1, verbose=10 if self.verbose else 0)(
            delayed(self._fit_estimator)(X, y, sample_indices, feature_indices, seed)
            for sample_indices, feature_indices, seed in zip(sample_indices_list, feature_indices_list, seeds)
        )
        return self

    def predict(self, X):
        X = np.asarray(X)
        predictions = np.array(Parallel(n_jobs=-1)(
            delayed(lambda est: est['estimator'].predict(X))(est) for est in self.estimators_
        )).T
        return np.round(np.mean(predictions, axis=1)).astype(int)

    def predict_proba(self, X):
        X = np.asarray(X)
        probas = np.mean(np.array(Parallel(n_jobs=-1)(
            delayed(lambda est: est['estimator'].predict_proba(X))(est) for est in self.estimators_
        )), axis=0)
        return probas
        
    def visualize(self, X, y, figsize=(10, 10)):
        """
        Visualize the ensemble decision boundaries.

        Parameters:
        -----------
        X : array-like, shape (n_samples, 2)
            The input samples
        y : array-like, shape (n_samples,)
            The target values
        figsize : tuple, default=(10, 10)
            Figure size
            
        Returns:
        --------
        fig : matplotlib figure
            The visualization figure
        """
        X = np.asarray(X)
        y = np.asarray(y)
        
        # Create a figure
        fig, ax = plt.subplots(figsize=figsize)
        
        # Plot data points
        ax.scatter(X[y == 0, 0], X[y == 0, 1], c='royalblue', label='Class 0', alpha=0.7)
        ax.scatter(X[y == 1, 0], X[y == 1, 1], c='coral', label='Class 1', alpha=0.7)
        
        # Create a mesh grid to visualize decision boundaries
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                             np.arange(y_min, y_max, 0.02))
        
        # Plot decision boundaries for each estimator
        for i, estimator_dict in enumerate(self.estimators_):
            estimator = estimator_dict['estimator']
            
            # Plot the spline curve
            x_curve = np.linspace(x_min, x_max, 100)
            y_curve = estimator.spline(x_curve)
            ax.plot(x_curve, y_curve, 'k-', alpha=0.3, linewidth=1)
            
        # Plot the ensemble decision boundary
        mesh_points = np.c_[xx.ravel(), yy.ravel()]
        Z = self.predict(mesh_points)
        Z = Z.reshape(xx.shape)
        
        # Plot the decision boundary with contour
        contour = ax.contour(xx, yy, Z, levels=[0.5], colors='k', linewidths=2)
        
        # Plot the probability heatmap
        proba = self.predict_proba(mesh_points)[:, 1]
        proba = proba.reshape(xx.shape)
        contourf = ax.contourf(xx, yy, proba, levels=np.linspace(0, 1, 11), 
                               cmap='RdBu_r', alpha=0.3)
        
        # Add colorbar
        cbar = plt.colorbar(contourf, ax=ax)
        cbar.set_label('Probability of Class 1')
        
        # Set labels and title
        ax.set_xlabel('Feature 1')
        ax.set_ylabel('Feature 2')
        ax.set_title('Ensemble SMPA Decision Boundary')
        ax.legend()
        
        return fig

In [4]:
def generate_moon_dataset(n_points=400, noise=0.2):
    X, y = make_moons(n_samples=n_points, random_state=7, noise=noise)
    return X, y

class ClassifierComparison:
    def __init__(self, X, y, random_state=1):
        self.X = X
        self.y = y
        self.random_state = random_state
        # We won't create a single split upfront anymore
        # but will keep track of class distribution for reference
        self.grids = {
            'smpa': {
                'learning_rate': [0.001, 0.005],
                'epochs': [300],
                'n_control_points': [7, 8, 9, 10],
                'decay_factor': [0.99, 0.999],
                'lambda_scaling': ['log', 'sqrt'],
                'patience' : [5, 10],
                'n_estimators' : [5, 10, 50],
                'spline_type' : ['cubic']
            },
            'svm': {
                'C': [0.1, 1, 10, 50, 100],
                'gamma': ['scale', 'auto', 0.1, 0.01],
                'kernel': ['rbf']
            },
            'rf': {
                'n_estimators': [100, 200],
                'max_depth': [10, 20],
                'min_samples_split': [2, 5]
            },
            'dt': {
                'max_depth': [5, 10, 20],
                'min_samples_split': [2, 5]
            }
        }
        # Define scaler types but don't fit them yet
        self.scaler_types = {
            'smpa': MinMaxScaler(feature_range=(-100, 100)),
            'svm': StandardScaler(),
            'rf': StandardScaler(),
            'dt': StandardScaler()
        }

    def create_train_test_split(self, seed):
        """Create a fresh train-test split with the given random seed"""
        X_train, X_test, y_train, y_test = train_test_split(
            self.X, self.y, test_size=0.2, random_state=seed, stratify=self.y
        )
        print(f"Train Class Dist - {np.bincount(y_train)}")
        print(f"Test Class Dist - {np.bincount(y_test)}")
        return X_train, X_test, y_train, y_test

    def scale_data(self, X_train, X_test, classifier_type):
        """Scale data using a fresh scaler for the given classifier type"""
        # Create a new scaler instance of the appropriate type
        scaler = self.scaler_types[classifier_type].__class__(**self.scaler_types[classifier_type].get_params())
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        return X_train_scaled, X_test_scaled

    def grid_search(self, X_train_scaled, y_train, classifier_type='smpa', seed=9):
        """Perform grid search with the provided scaled training data"""
        classifiers = {
            'smpa': EnsembleSMPA(random_state=seed),
            'svm': SVC(random_state=seed),
            'rf': RandomForestClassifier(random_state=seed),
            'dt': DecisionTreeClassifier(random_state=seed),
        }
        clf = classifiers[classifier_type]
        param_grid = self.grids[classifier_type]
        cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
        grid_search = GridSearchCV(clf, param_grid, cv=cv, scoring='accuracy', n_jobs=10, verbose=3)
        grid_search.fit(X_train_scaled, y_train)
        return grid_search

        # Modify the stability_test method in ClassifierComparison class
    def stability_test(self, classifier_type='smpa', n_runs=5, save_plots=True, show_plots=False):
        scores = []
        best_params_list = []
        
        for seed in range(n_runs):
            print(f"\nRun {seed} for {classifier_type}...")
            
            # Create a fresh train-test split for each run
            X_train, X_test, y_train, y_test = self.create_train_test_split(seed)
            
            # Scale the data for this specific split
            X_train_scaled, X_test_scaled = self.scale_data(X_train, X_test, classifier_type)
            
            # Grid search on this fresh train split with newly scaled data
            grid_search = self.grid_search(X_train_scaled, y_train, classifier_type, seed)
            clf = grid_search.best_estimator_
            best_params_list.append(grid_search.best_params_)
            print(f"Run {seed} Best Params: {grid_search.best_params_}")
            
            try:
                y_pred = clf.predict(X_test_scaled)
                score = accuracy_score(y_test, y_pred)
                scores.append(score)
                
                # if classifier_type == 'smpa':
                #     # Plot with the same scaled data used for training
                #     try:
                #         fig = clf.visualize(X_train_scaled, y_train)
                #         if save_plots:
                #             # Save the figure instead of displaying it
                #             plt.savefig(f"{classifier_type}_run_{seed}.png", dpi=100)
                #         if show_plots:
                #             plt.show()
                #         # Always close the figure to free memory
                #         plt.close(fig)
                #     except Exception as e:
                #         print(f"🚨 Error in visualization for run {seed}: {e}")
                
                print(f"Run {seed} Score: {score:.4f}")
            except Exception as e:
                print(f"🚨 Error in run {seed}: {e}")
                scores.append(0)
        
        return {
            'mean_score': np.mean(scores),
            'std_score': np.std(scores),
            'scores': scores,
            'best_params_list': best_params_list
        }
    def statistical_significance_test(self, baseline_scores_dict, target='smpa'):
        target_scores = baseline_scores_dict[target]['scores']
        for clf_type, results in baseline_scores_dict.items():
            if clf_type != target:
                t_stat, p_val = stats.ttest_ind(target_scores, results['scores'])
                print(f"\n🔬 {target} vs. {clf_type}:")
                print(f"T-Statistic: {t_stat:.4f}")
                print(f"P-Value: {p_val:.4f}")
                print("🏆 Significant difference!" if p_val < 0.05 else "🤝 No significant difference.")

In [5]:
import types

X, y = generate_moon_dataset(n_points=400, noise=0.3)
comparison = ClassifierComparison(X, y)

# Run all classifiers on same split
classifiers = ['smpa', 'svm', 'rf', 'dt']
results = {}

# Reduce the computation load for grid search
# Modify the GridSearchCV in the grid_search method
def grid_search_modified(self, X_train_scaled, y_train, classifier_type='smpa', seed=0):
    """Perform grid search with the provided scaled training data"""
    classifiers = {
        'smpa': EnsembleSMPA(random_state=seed),
        'svm': SVC(random_state=seed),
        'rf': RandomForestClassifier(random_state=seed),
        'dt': DecisionTreeClassifier(random_state=seed),
    }
    clf = classifiers[classifier_type]
    param_grid = self.grids[classifier_type]
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)
    # Limit the number of parallel jobs
    grid_search = GridSearchCV(clf, param_grid, cv=cv, scoring='accuracy', n_jobs=-1, verbose=3)
    grid_search.fit(X_train_scaled, y_train)
    return grid_search

# Replace the original method with the modified one
comparison.grid_search = types.MethodType(grid_search_modified, comparison)

for clf_type in classifiers:
    # Set save_plots=True and show_plots=False to avoid VS Code renderer issues
    results[clf_type] = comparison.stability_test(clf_type, n_runs=5, save_plots=True, show_plots=False)

for clf_type, res in results.items():
    print(f"\n📊 {clf_type.upper()} Stability:")
    print(f"Mean Score: {res['mean_score']:.4f}")
    print(f"Score Std Dev: {res['std_score']:.4f}")

comparison.statistical_significance_test(results, target='smpa')


Run 0 for smpa...
Train Class Dist - [160 160]
Test Class Dist - [40 40]
Fitting 3 folds for each of 192 candidates, totalling 576 fits
[CV 1/3] END decay_factor=0.99, epochs=300, lambda_scaling=log, learning_rate=0.001, n_control_points=7, n_estimators=5, patience=10, spline_type=cubic;, score=0.860 total time=  56.6s
[CV 3/3] END decay_factor=0.99, epochs=300, lambda_scaling=log, learning_rate=0.001, n_control_points=7, n_estimators=5, patience=10, spline_type=cubic;, score=0.811 total time=  58.0s
[CV 1/3] END decay_factor=0.99, epochs=300, lambda_scaling=log, learning_rate=0.001, n_control_points=7, n_estimators=5, patience=5, spline_type=cubic;, score=0.860 total time=  59.4s
[CV 3/3] END decay_factor=0.99, epochs=300, lambda_scaling=log, learning_rate=0.001, n_control_points=7, n_estimators=5, patience=5, spline_type=cubic;, score=0.821 total time=  59.7s
[CV 2/3] END decay_factor=0.99, epochs=300, lambda_scaling=log, learning_rate=0.001, n_control_points=7, n_estimators=5, pati

KeyboardInterrupt: 